In [13]:
"""
load data
"""
%reload_ext autoreload
import numpy as np
import csvio
import copy
import random
import data_processing as dp

motions, headers = csvio.load_data('./input_data/train/train.csv')

bow_motions = motions[0:11]
wav_motions = motions[11:33]
run_motions = motions[33:45] + motions[56:67]
frt_motions = motions[45:56]
wlk_motions = motions[67:91]
bck_motions = motions[91:103]
rgt_motions = motions[103:115]
lft_motions = motions[115:127] 

model_settings = {
    "0_45_bow": [45, 2, bow_motions, np.linspace(0.9, 1.1, 15)],
    "1_45_wav": [45, 3, wav_motions, np.linspace(0.9, 1.1, 15)],
    "2_45_run": [45, 1, run_motions, np.linspace(0.9, 1.1, 15)],
    # "3_45_frt":  [45, 1, list(range(21)), frt_motions, np.linspace(0.8, 1.2, 15), None],
    # "4_45_wlk":  [45, 3, list(range(21)), wlk_motions, np.linspace(0.8, 1.1, 25), None],
    # "6_45_rgt":  [45, 1, list(range(21)), rgt_motions, np.linspace(0.9, 1.1, 25), None],

    # "0_15_bow":  [15, 3, list(range(21)), run_motions, np.linspace(0.9, 1.1, 15), None],
    # "1_15_wavr": [15, 3, list(range(21)), wavr_motions, np.linspace(0.9, 1.1, 15), None],
    # "2_15_run": [15, 3, list(range(21)), run_motions, np.linspace(0.9, 1.1, 15), None],
    # "3_15_frt": [15, 3, list(range(21)), frt_motions, np.linspace(0.9, 1.1, 15), None],
    # "4_15_wlk": [15, 3, list(range(21)), wlk_motions, np.linspace(0.9, 1.1, 15), None],
    # "5_15_bck": [15, 3, list(range(21)), bck_motions, np.linspace(0.9, 1.1, 15), None],
    # "6_15_rgt": [15, 3, list(range(21)), rgt_motions, np.linspace(0.9, 1.1, 15), None],
    # "7_15_lft": [15, 3, list(range(21)), lft_motions, np.linspace(0.9, 1.1, 15), None],
}

In [14]:
import random
import torch
import torch.nn as nn
import numpy as np
import interpolation
from model import TransMotion
import prediction as pred
import graph
import math
import dataset

torch.manual_seed(0)
np.random.seed(0)

def train(source, target):
    """
    source: (data_size, window, feature)
    target: (data_size, window, feature)
    """
    model.train()
    total_loss = 0.
    for _ in range(0, len(source) - batch_size, batch_size):
        data, targets = dataset.get_both_batch(source, target, batch_size, input_window)

        optimizer.zero_grad()
        output = model(data)

        if additional_loss:
            exist_frames = list(range(0, input_window, interval))
            loss = criterion(output, targets) + criterion(output[exist_frames], targets[exist_frames])
        else:
            loss = criterion(output, targets)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(range(0, len(source) - 1, batch_size))


def evaluate(eval_model, source, target):
    eval_batch_size = min(64, len(source))
    eval_model.eval()
    with torch.no_grad():
        src, tgt = dataset.get_both_batch(source, target, eval_batch_size, input_window)
        out = eval_model(src)

        index = random.randint(0, out.shape[2] - 1)
        marker = index // 3
        axis = index % 3
        filename = f"eval_{epoch}_{marker}_{axis}.png"
        graph.plot_name(src.cpu().detach().numpy()[:, 0, index],
                   out.cpu().detach().numpy()[:, 0, index],
                   tgt.cpu().detach().numpy()[:, 0, index],
                   filename)

        loss = criterion(out, tgt)

    return loss.item()

batch_size = 8
device = torch.device("cuda")
additional_loss = False
X, Y, Z = 0, 1, 2


In [15]:
for name in model_settings:
    epochs = 20
    interval, sections, motion_list, rates = model_settings[name]
    input_window = interval * sections + 1
    train_motions, valid_motions = dataset.get_train_valid(motion_list)

    for model_num in range(3):
        model_path = f"model/{name}_{model_num}.pth"

        best_valid_loss = float("inf")
        model = TransMotion(interval, d_input=63).to(device).double()
        lr = 0.005
        criterion = nn.MSELoss()
        optimizer = torch.optim.RAdam(model.parameters(), lr=lr, weight_decay=0.00001)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.995)

        print(f"====== {model_path} ======")
        train_source, train_target = dataset.make_dataset(train_motions, interval, input_window, rates)
        valid_source, valid_target = dataset.make_dataset(valid_motions, interval, input_window, rates)
        train_loss_list = []
        valid_loss_list = []
        for epoch in range(1, epochs + 1):
            train_loss = train(train_source, train_target)
            valid_loss = evaluate(model, valid_source, valid_target)
            train_loss_list.append(train_loss)
            valid_loss_list.append(valid_loss)
            if valid_loss < best_valid_loss:
                best_valid_loss = valid_loss
                torch.save(model.state_dict(), model_path)
            print(f'epoch {epoch:3d} | train {train_loss:5.5f} | valid {valid_loss:5.5f}')
            scheduler.step()
            graph.plot_loss(train_loss_list, valid_loss_list)
        print("")
    

train_motions: 20
valid_motions: 3
====== model/2_45_run_0.pth ======
data: torch.Size([4395, 46, 63])
data: torch.Size([1429, 46, 63])
| epoch   1 | train loss 0.06163 | valid loss 0.02669 | lr 0.00500 | best
| epoch   2 | train loss 0.03088 | valid loss 0.02466 | lr 0.00498 | best
| epoch   3 | train loss 0.02824 | valid loss 0.02925 | lr 0.00495 |
| epoch   4 | train loss 0.02613 | valid loss 0.02671 | lr 0.00493 |
| epoch   5 | train loss 0.02436 | valid loss 0.03702 | lr 0.00490 |
| epoch   6 | train loss 0.02431 | valid loss 0.02499 | lr 0.00488 |
| epoch   7 | train loss 0.02196 | valid loss 0.02724 | lr 0.00485 |
| epoch   8 | train loss 0.02194 | valid loss 0.02756 | lr 0.00483 |
| epoch   9 | train loss 0.02162 | valid loss 0.03121 | lr 0.00480 |
| epoch  10 | train loss 0.01967 | valid loss 0.03170 | lr 0.00478 |
| epoch  11 | train loss 0.01971 | valid loss 0.02286 | lr 0.00476 | best
| epoch  12 | train loss 0.02005 | valid loss 0.02602 | lr 0.00473 |
| epoch  13 | train l

In [16]:
%load_ext autoreload
%autoreload 2

import data_processing as dp
import matplotlib.pyplot as plt

def search_best_weights(model, test_motions, name):
    model0 = copy.deepcopy(model)
    model1 = copy.deepcopy(model)
    model2 = copy.deepcopy(model)
    model0.load_state_dict(torch.load(f"model/{name}_0.pth"))
    model1.load_state_dict(torch.load(f"model/{name}_1.pth"))
    model2.load_state_dict(torch.load(f"model/{name}_2.pth"))

    # all motions
    pred_data0 = []
    pred_data1 = []
    pred_data2 = []
    lerp_data = []
    for test_motion in test_motions:
        if test_motion.data.shape[0] < input_window:
            pred_data0.append(0.0)
            pred_data1.append(0.0)
            pred_data2.append(0.0)
            lerp_data.append(0.0)
            continue
        source_data, target_data = dataset.make_predict_data(test_motion, interval)
        source_data = dp.lost(source_data, interval)

        pred_data0.append(pred.predict(model0, source_data, input_window, interval))
        pred_data1.append(pred.predict(model1, source_data, input_window, interval))
        pred_data2.append(pred.predict(model2, source_data, input_window, interval))
        lerp_data.append(interpolation.linear_interpolate(source_data, interval))
    
    best_weights = np.zeros((21, 4))
    min_loss = np.full((21), float("inf"))
    for marker in range(21):
        for w0 in range(0, 11):
            for w1 in range(0, 11 - w0):
                for w2 in range(0, 11 - w0 - w1):
                    w3 = 10 - w0 - w1 - w2
                    pred_loss = 0
                    for index, test_motion in enumerate(test_motions):
                        if test_motion.data.shape[0] < input_window:
                            continue
                        source_data, target_data = dataset.make_predict_data(test_motion, interval)
                        data0 = pred_data0[index][:, marker]
                        data1 = pred_data1[index][:, marker]
                        data2 = pred_data2[index][:, marker]
                        data3 = lerp_data[index][:, marker]
                        pred_data = (data0 * w0 + data1 * w1 + data2 * w2 + data3 * w3) / 10.0
                        pred_loss += dp.calc_loss(pred_data, target_data[:, marker])
                    if pred_loss < min_loss[marker]:
                        min_loss[marker] = pred_loss
                        best_weights[marker] = [w0, w1, w2, w3]
    return best_weights

def predict_with_weights(model, source_data, weights, markers, name):
    model0 = copy.deepcopy(model)
    model1 = copy.deepcopy(model)
    model2 = copy.deepcopy(model)
    model0.load_state_dict(torch.load(f"model/{name}_0.pth"))
    model1.load_state_dict(torch.load(f"model/{name}_1.pth"))
    model2.load_state_dict(torch.load(f"model/{name}_2.pth"))
    
    pred_data0 = pred.predict(model0, source_data, input_window, interval)
    pred_data1 = pred.predict(model1, source_data, input_window, interval)
    pred_data2 = pred.predict(model2, source_data, input_window, interval)
    lerp_data = interpolation.linear_interpolate(source_data, interval)
    
    pred_data = np.zeros(source_data.shape)
    for marker in range(21):
        w0, w1, w2, w3 = weights[marker]
        pred_data[:, marker] = (pred_data0[:, marker] * w0 + 
                                pred_data1[:, marker] * w1 + 
                                pred_data2[:, marker] * w2 + 
                                lerp_data[:, marker] * w3) / 10.0
        # for axis in range(3):
        #     plt.figure(figsize=(12, 8))
        #     plt.plot(pred_data0[:, marker, axis], label="pred_data0", c='r', linestyle='dashed')
        #     plt.plot(pred_data1[:, marker, axis], label="pred_data1", c='r', linestyle='dashed')
        #     plt.plot(pred_data2[:, marker, axis], label="pred_data2", c='r', linestyle='dashed')
        #     plt.plot(lerp_data[:, marker, axis], label="lerp_data", c='g', linestyle='dashed')
        #     plt.plot(pred_data[:, marker, axis], label="pred_data", c='b')
        #     plt.legend()
        #     # plt.savefig(f"graph/{name}_{marker}_{axis}.png")
        #     plt.close()

    return pred_data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
"""
test
"""

all_lerp_motions = []
all_pred_motions = []
for name in model_settings:
    interval, sections, markers, motion_list, rates, filter_setting = model_settings[name]
    input_window = interval * sections + 1
    
    model = TransMotion(interval).to(device).double()

    print(f"====== {name} ======")
    _, valid_motions = dataset.get_train_valid(motion_list)

    # TODO: motion list
    weights = search_best_weights(model, valid_motions, name)
    print(f"weights: {weights}")
    # === predict ===
    lerp_motions = copy.deepcopy(valid_motions)
    pred_motions = copy.deepcopy(valid_motions)
    total_loss = 0.0
    for index, valid_motion in enumerate(valid_motions):
        if valid_motion.data.shape[0] < input_window:
            continue
        source_data, target_data = dataset.make_predict_data(valid_motion, interval)
        source_data = dp.lost(source_data, interval)

        print("motion:", valid_motion.name, valid_motion.data.shape)

        lerp_data = interpolation.linear_interpolate(source_data, interval)
        pred_data = predict_with_weights(model, source_data, weights, markers, name)
        
        lerp_loss = dp.calc_loss(lerp_data[:, markers], target_data[:, markers])
        pred_loss = dp.calc_loss(pred_data[:, markers], target_data[:, markers])

        # for marker in range(21):
        #     for axis in range(3):
        #         plt.figure(figsize=(12, 8))
        #         plt.plot(lerp_data[:, marker, axis], label="lerp_data", c='g', linestyle='dashed')
        #         plt.plot(pred_data[:, marker, axis], label="pred_data", c='r')
        #         plt.plot(target_data[:, marker, axis], label="target_data", c='b')
        #         plt.legend()
        #         plt.show()
        #         plt.close()


        print(f'\tlerp loss: {lerp_loss:5.5f}')
        print(f'\tpred loss: {pred_loss:5.5f}')
        total_loss += pred_loss

        lerp_motions[index].data = lerp_data
        pred_motions[index].data = pred_data

    print("total_loss:", total_loss)
    all_lerp_motions = all_lerp_motions + lerp_motions
    all_pred_motions = all_pred_motions + pred_motions

csvio.write_csv("output_data/test_lerp.csv", headers, all_lerp_motions)
csvio.write_csv("output_data/test_pred.csv", headers, all_pred_motions)

====== 2_45_run ======
train_motions: 20
valid_motions: 3
weights: [[7. 0. 0. 3.]
 [6. 0. 0. 4.]
 [6. 0. 0. 4.]
 [4. 0. 2. 4.]
 [2. 1. 4. 3.]
 [5. 0. 1. 4.]
 [6. 0. 0. 4.]
 [7. 1. 2. 0.]
 [4. 0. 5. 1.]
 [4. 0. 2. 4.]
 [4. 0. 2. 4.]
 [5. 1. 3. 1.]
 [4. 2. 4. 0.]
 [7. 0. 0. 3.]
 [5. 0. 3. 2.]
 [5. 2. 2. 1.]
 [6. 2. 1. 1.]
 [7. 0. 0. 3.]
 [6. 1. 1. 2.]
 [4. 2. 2. 2.]
 [4. 1. 3. 2.]]
motion: 082 (141, 21, 3)
	lerp loss: 88.64927
	pred loss: 8.68404
motion: 083 (141, 21, 3)
	lerp loss: 81.93764
	pred loss: 21.25569
motion: 084 (133, 21, 3)
	lerp loss: 76.30894
	pred loss: 20.29217
total_loss: 50.23189692463438


In [ ]:
"""
hard
"""

motions, header = csvio.load_data('./input_data/test/test_hard.csv')
device = torch.device("cuda")

methods = {
    "013": "0_45_bow",     # bow
    "022": "1_45_wavr",     # wav right
    "032": "1_45_wavb",     # wav both
    "049": "2_45_run", # run
    # "059": "lerp",     # frt
    "059": "3_45_frt", # frt
    "078": "2_45_run", # run
    "098": "4_45_wlk", # walk
    "101": "4_45_wlk", # walk
    "113": "4_45_wlk", # walk
    "134": "6_45_rgt", # right
}

for motion in motions:
    source_data = motion.data
    if motion.name != "059":
        continue

    method = methods[motion.name]
    print(f"motion: {motion.name} {method} {source_data.shape}")


    if method == "lerp":
        output_data = interpolation.linear_interpolate(source_data, interval)
    elif method == "quad":
        output_data = interpolation.quadratic_interpolate(source_data, interval)
    else:
        interval, sections, markers, motion_list, rates, filter_setting = model_settings[method]
        input_window = interval * sections + 1
        d_input = len(markers) * 3
        model = TransMotion(interval).to(device).double()
        _, valid_motions = dataset.get_train_valid(motion_list)

        weights = search_best_weights(model, valid_motions, method)
        output_data = predict_with_weights(model, source_data, weights, markers, method)

        for marker in range(21):
            for axis in range(3):
                plt.figure(figsize=(12, 8))
                plt.plot(source_data[:, marker, axis], label="lerp_data", c='g', linestyle='dashed')
                plt.plot(output_data[:, marker, axis], label="pred_data", c='r')
                plt.legend()
                plt.show()
                plt.close()
        

    motion.data = output_data

csvio.write_csv("./output_data/test_hard.csv", header, motions)



In [ ]:
"""
normal
"""

motions, header = csvio.load_data('./input_data/test/test_normal.csv')
device = torch.device("cuda")

methods = {
    "005": "quad",  # bow
    # "028": "lerp",  # wave_hands
    "028": "1_15_wavr",  # wave_hands
    "036": "3_15_frt",  # front_hands
    "067": "3_15_frt",  # front_hands
    "079": "2_15_run", # run
    "085": "4_15_wlk", # walk
    "122": "5_15_bck", # back
    "130": "6_15_rgt", # right
    "144": "7_15_lft", # left
    "155": "7_15_lft", # left
}


for motion in motions:
    source_data = motion.data

    method = methods[motion.name]
    print(f"motion: {motion.name} {method} {source_data.shape}")

    if method == "lerp":
        output_data = interpolation.linear_interpolate(source_data, interval)
    elif method == "quad":
        output_data = interpolation.quadratic_interpolate(source_data, interval)
    else:
        interval, sections, markers, motion_list, rates, filter_setting = model_settings[method]
        input_window = interval * sections + 1
        d_input = len(markers) * 3
        model = TransMotion(interval, d_input=d_input).to(device).double()
        _, valid_motions = dataset.get_train_valid(motion_list)

        weights = search_best_weights(model, valid_motions, markers, method)
        output_data = predict_with_weights(model, source_data, weights, markers, method)

    motion.data = output_data

csvio.write_csv("./output_data/test_normal.csv", header, motions)

